In [4]:
import requests

userAgent = {"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"}
getDownload = lambda url: requests.get(url, headers=userAgent)

In [5]:
import re

getCleanText = lambda item: re.sub(r"[\s]{2,}", " ", item.strip().replace(",", " "))

In [6]:
from bs4 import BeautifulSoup

url = "https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=136900&target=after&page="

pageNo = 1
maxPageNo = 10
maxPageNo = 2
lastReviewNo = 0

saveFilePath = "./어벤져스.txt"

fp = open(saveFilePath, "w", encoding="utf-8")
fp.close()

html = getDownload(url+str(pageNo))

while html.status_code == 200 and pageNo < maxPageNo:
    dom = BeautifulSoup(html.text, "html.parser")
    pointList = dom.select(".list_netizen tbody > tr")
    
    fp = open(saveFilePath, "a", encoding="utf-8")
    
    for pointInfo in pointList:
        pointItem = [point for point in pointInfo.select("td")]
        pointID = getCleanText(pointItem[0].text)
        pointScore = getCleanText(pointItem[2].text)
        pointMovie = getCleanText(pointItem[3].select_one("a").text)
        pointComment = getCleanText(pointItem[3].contents[4])
        pointRegData = getCleanText(pointItem[4].contents[-1])
        
        if lastReviewNo == pointID:
            continue
        else:
            fp.write("{0},{1},{2},{3},{4}\n".format(pointID, pointScore, pointMovie, pointComment, pointRegData))
        
    fp.close()
    
    lastReviewNo = pointID
        
    pageNo += 1
    html = getDownload(url+str(pageNo))
    
    if pageNo % 100 == 0:
        print("{0} pages are crawled.".format(pageNo))

In [8]:
import numpy as np
import pandas as pd

data = pd.read_csv("./어벤져스.txt", engine='python', header=None, names=["title", "score", "review", "date"], encoding='utf-8')
data = data[data.review.notnull()]
data = data.drop_duplicates()

In [10]:
data.count()

title     10
score     10
review    10
date      10
dtype: int64

In [9]:
from collections import defaultdict
from konlpy.tag import Komoran

ma = Komoran()

documents = defaultdict(list)
vocabulary = list()

for i, review in zip(data.index, data.review):
    for term in [_ for _ in ma.nouns(review) if 1 < len(_) < 8]:
        documents[i].append(term)
        vocabulary.extend(documents[i])
        
vocabulary = list(set(vocabulary))